<div dir="rtl">

<b style="font-size: 30px;">پشت پرده پایپ‌لاین در ترنسفورمرها</b>

</div>

<div dir="rtl">

<p>
در فصل اول با تابع <code>pipeline()</code> آشنا شدیم.
</p>

</div>


In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
classifier(
    [
        "I've been waiting for a HuggingFace course my whole life.",
        "I hate this so much!",
    ]
)

[{'label': 'POSITIVE', 'score': 0.9598047137260437},
 {'label': 'NEGATIVE', 'score': 0.9994558095932007}]

<div dir="rtl">

<p>
این پایپ‌لاین سه مرحله را در کنار هم قرار می‌دهد: پیش‌پردازش، عبور دادن ورودی‌ها از درون مدل، و پس‌پردازش.
</p>

</div>

---

<div dir="rtl">

<b style="font-size: 18px;"> پیش‌پردازش با توکنایزر</b>

<p>
مدل‌های ترنسفورمر نمی‌توانند متن خام را مستقیماً پردازش کنند، بنابراین ابتدا باید متن را به عدد تبدیل کنیم. این کار با <b>توکنایزر</b> انجام می‌شود که وظیفه‌اش:
</p>

<ul>
  <li>شکستن متن به توکن‌ها (کلمات، زیرکلمات یا نمادها)</li>
  <li>تبدیل هر توکن به عدد</li>
  <li>افزودن ورودی‌های اضافی لازم برای مدل</li>
</ul>

<p>
تمام این مراحل باید دقیقاً مطابق با آموزش اولیهٔ مدل انجام شود. برای همین، با استفاده از کلاس <code>AutoTokenizer</code> و متد <code>from_pretrained()</code>، توکنایزر مناسب مدل را از Model Hub بارگذاری می‌کنیم.
</p>

<p>
برای مدل پیش‌فرض تحلیل احساسات (<code>distilbert-base-uncased-finetuned-sst-2-english</code>)، این کد اجرا می‌شود.
</p>

</div>


In [ ]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

<div dir="rtl">

<b style="font-size: 18px;"> تبدیل ورودی به تنسور</b>

<p>
پس از گرفتن توکنایزر، می‌توانیم جمله‌ها را به آن بدهیم و یک خروجی آماده برای مدل دریافت کنیم. تنها کار باقی‌مانده این است که شناسه‌های ورودی (<code>input IDs</code>) را به <b>تنسور</b> تبدیل کنیم.
</p>

<p>
 <b>مدل‌های ترنسفورمر فقط تنسورها را به‌عنوان ورودی می‌پذیرند</b>، اما نگران فریم‌ورک (مثل PyTorch یا Flax) نباشید — کتابخانه  Transformers خودش آن را مدیریت می‌کند.
</p>

<p>
اگر با تنسورها آشنایی ندارید، آن‌ها را مثل آرایه‌های NumPy تصور کنید:
</p>

<ul>
  <li>عدد (۰D)</li>
  <li>بردار (۱D)</li>
  <li>ماتریس (۲D) یا بیشتر</li>
</ul>

<p>
برای تعیین نوع تنسور، از آرگومان <code>return_tensors</code> استفاده می‌کنیم (مثلاً <code>"pt"</code> برای PyTorch).
</p>

</div>


In [ ]:
raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
print(inputs)

{
    'input_ids': tensor([
        [  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172, 2607,  2026,  2878,  2166,  1012,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,     0,     0,     0,     0,     0,     0]
    ]), 
    'attention_mask': tensor([
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
    ])
}

In [ ]:
from transformers import AutoModel

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModel.from_pretrained(checkpoint)

In [ ]:
outputs = model(**inputs)
print(outputs.last_hidden_state.shape)

torch.Size([2, 16, 768])

In [ ]:
from transformers import AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)

In [ ]:
print(outputs.logits.shape)

torch.Size([2, 2])

In [ ]:
print(outputs.logits)

tensor([[-1.5607,  1.6123],
        [ 4.1692, -3.3464]], grad_fn=<AddmmBackward>)

In [ ]:
import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

tensor([[4.0195e-02, 9.5980e-01],
        [9.9946e-01, 5.4418e-04]], grad_fn=<SoftmaxBackward>)

In [ ]:
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}